# Calculations without CuPy or Cudf

In [52]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians, degrees, acos, asin, pi, atan, isnan
import progressbar
import time
#import cupy as cp

In [45]:
PROBE_DATA_PATH = "./input/Partition6467ProbePoints.csv"
LINK_DATA_PATH="./input/Partition6467LinkData.csv"
OUTPUT_MATCHED_PATH="./output/MatchedProbePoint.csv"
OUTPUT_MATCHED_PATH_WITH_DIRECTION="./output/MatchedProbePointWithDirection.csv"
OUTPUT_SLOPE_DATA_PATH="./output/SlopeData.csv"
EVALUATE_SLOPE_DATA_PATH="./output/EvaluateSlopeData.csv"

In [8]:
probe_col_names=["sampleID", "dateTime", "sourceCode", "latitude", "longitude", "altitude", "speed", "heading"]
probe_data = pd.read_csv(PROBE_DATA_PATH,header=None,names=probe_col_names)

In [77]:
link_col_names=["linkPVID", "refNodeID", "nrefNodeID", "length", "functionalClass", "directionOfTravel", "speedCategory", "fromRefSpeedLimit", "toRefSpeedLimit", "fromRefNumLanes", "toRefNumLanes", "multiDigitized", "urban", "timeZone", "shapeInfo", "curvatureInfo", "slopeInfo"]
link_data = pd.read_csv(LINK_DATA_PATH,header=None,names=link_col_names)
link_data['slopeInfo']=link_data['slopeInfo'].astype(str)

- No null values in probe data
- In LinkData: 159289 curvature info missing, 146572 slope info missing




### Preprocess Link Data

In [10]:
# A function that process the shapeInfo and returns an array of shape points (ref,shape points, non-ref)
def process_shapeInfo(df):
  bar = progressbar.ProgressBar(maxval=df.shape[0], \
  widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
  bar.start()
  i=0
  shape_points = []
  shapeInfo = df['shapeInfo'];
  
  for data in shapeInfo:
    points = data.split('|');
    temp_list=[]
    for point in points:
      temp = point.split('/')
      coord = (float(temp[0]),float(temp[1]))
      temp_list.append(coord)

    shape_points.append(temp_list)
    i+=1
    bar.update(i)

  bar.finish()
  return shape_points
  

### Calculate midpoint for each link

Formula referenced from [Movable Type Scripts](http://www.movable-type.co.uk/scripts/latlong.html)

In [11]:
def mid_point(lat1,lon1,lat2,lon2):
    # convert lats and lons to radian 
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    # calculate bearings
    Bx = cos(lat2) * cos(lon2-lon1)
    By = cos(lat2) * sin(lon2-lon1)
    
    # calculate mid point
    latMid = atan2(sin(lat1) + sin(lat2), sqrt((cos(lat1)+Bx)**2+ By**2))
    lonMid = lon1 + atan2(By, cos(lat1) + Bx)

    # convert to degrees
    latMid = degrees(latMid)
    lonMid = degrees(lonMid)
    
    return (latMid,lonMid)

### To calculate distance between two points


Uses the **Haversine** formula to calculate the great-circle distance between two points – that is, the shortest distance over the earth’s surface
[stackoverflow answer](https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude/43211266#43211266)

In [12]:
def point_distance(lat1,lon1,lat2,lon2):
  # approximate radius of earth in km
  R = 6371.0

  lat1 = radians(lat1)
  lon1 = radians(lon1)
  lat2 = radians(lat2)
  lon2 = radians(lon2)

  dlon = lon2 - lon1
  dlat = lat2 - lat1

  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  distance = R * c
  return distance

### To calculate smallest distance between a point and a line

Formula referenced from [stackoverflow](https://stackoverflow.com/questions/20231258/minimum-distance-between-a-point-and-a-line-in-latitude-longitude) as well as [this website](http://www.movable-type.co.uk/scripts/latlong.html)

In [13]:
def min_distance_point_to_line(lat1,lon1,lat2,lon2,lat3,lon3):

  # Supposed we have a line defined by A and B, and a point C
  # A and B = lat1,lon1,lat2,lon2
  # point C = lat3,lon3
  
  R = 6371.0
  # 1. Find the bearing from A to C and A to B
  # Bearing A to C
  y = sin(lon3 - lon1) * cos(lat3)
  x = cos(lat1) * sin(lat3) - sin(lat1) * cos(lat3) * cos(lat3 - lat1)
  bearing1 = atan2(y, x)
  bearing1 = (bearing1*180/pi+360) % 360 #degrees
  
  # Bearing A to B
  y2 = sin(lon2 - lon1) * cos(lat2)
  x2 = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(lat2 - lat1)
  bearing2 = atan2(y2, x2)
  bearing2 = (bearing2*180/pi+360) % 360  #degrees

  lat1Rads = radians(lat1)
  lat3Rads = radians(lat3)
  dLon = radians(lon3 - lon1)

  # 2. Find distance from A to C using spherical law of consines
  distanceAC = acos(sin(lat1Rads) * sin(lat3Rads)+cos(lat1Rads)*cos(lat3Rads)*cos(dLon)) * R

  # 3. Find cross-track distance
  min_distance = abs(asin(sin(distanceAC/R)*sin(radians(bearing1)-radians(bearing2))) * R)

  return min_distance

### To search for the index of the closest link to the probe

In [14]:
# binary search the closest link to the probe with distance from bottom left of bounding box
def closest_link_index(probe_distance, link_distances, start, end):
    if start < end:
        mid = (start + end) // 2

        if probe_distance < link_distances[mid]:
            return closest_link_index(probe_distance, link_distances, start, mid-1)
        else:
            return closest_link_index(probe_distance, link_distances, mid+1, end)
    else:
        return start

<br></br>
### Map Matching

In [15]:
def map_matching(probe_data,link_data,threshold): 
  bar = progressbar.ProgressBar(maxval=probe_data.shape[0], \
  widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
  bar.start()
  j=0

  # change threshold to threshold / 2 to account for link distances that are greater and smaller than the probe distance
  threshold = threshold // 2
  
  # create a dataframe for the matchedpoints output
  matched_points = probe_data.copy()
    
  # add required columns in the dataframe, filled with default values
  matched_points['linkPVID']=0 # LinkID from link_data
  matched_points['direction']='F' # the direction the vehicle was travelling on thelink (F = from ref node, T = towards ref node)
  matched_points['distFromRef']=0.0 # the distance from the reference node to the map-matched probe point location on the link in decimal meters
  matched_points['distFromLink']=0.0 # the perpendicular distance from the map-matched probe point location on the link to the probe point in decimal meters

  # iterate through entire probe data, for each probe point, iterate through entire link data to find link with min distFromLink
  for probe in matched_points.itertuples():
    start_time = time.time()
    
    probe_point = (probe.latitude,probe.longitude)
    min_dist = float('inf')  # start with min dist as positive infinity
    
    # look for closest link
    closest_link = closest_link_index(probe.probe_distance, link_data['midpoint_distance'], 0, len(link_data))
    
    # define search area with threshold
    begin_search = closest_link - threshold if closest_link - threshold >= 0 else 0
    end_search = closest_link + threshold if closest_link + threshold <= len(link_data) else len(link_data)
    
    for link in link_data[begin_search:end_search].itertuples():
      shape_arr = link.shape_points

      # if no curvatureInfo exist for this link (meaning a straight link), use only the ref and non-ref point to calculate
      if pd.isnull(link.curvatureInfo):  
        ref= shape_arr[0] #ref node is at index 0
        non_ref= shape_arr[len(shape_arr)-1] # non-ref node is at the last index
        temp = min_distance_point_to_line(ref[0],ref[1],non_ref[0],non_ref[1],probe_point[0],probe_point[1])
        if temp < min_dist:
          min_dist = temp
          linkID = link.linkPVID
          min_link_ref = ref= shape_arr[0]
      
      # if curvatureInfo exist, use all provided points, calculate the min distance from probe point to each of the intermediate link segment
      else:
        # iterate through all pairs consecutive pairs of shape points and calculate the line-point distance for each pair
        for i in range(len(shape_arr)-1):
          p1 = shape_arr[i]
          p2 = shape_arr[i+1]
          temp = min_distance_point_to_line(shape_arr[i][0],shape_arr[i][1],shape_arr[i+1][0],shape_arr[i+1][0],probe_point[0],probe_point[1])
          if temp < min_dist:
            min_dist = temp
            linkID = link.linkPVID
            min_link_ref = ref= shape_arr[0]
      
    # set found min_distance and corresponding linkPVID for this probe
    matched_points.at[probe.Index,'linkPVID']=linkID
    matched_points.at[probe.Index,'distFromLink']=min_dist

    # calculate distFromRef (dist from probe to ref node)
    distFromRef = point_distance(probe_point[0],probe_point[1],min_link_ref[0],min_link_ref[1])
    matched_points.at[probe.Index,'distFromRef']=distFromRef
    j+=1
    bar.update(j)
    
    end_time = time.time()
    #print(f"Time for {j}: {end_time - start_time}")

  bar.finish()
  return matched_points



### Main Program

In [16]:
# process shape info
shape_points = process_shapeInfo(link_data)

if len(shape_points) == link_data.shape[0]: 
    # Add the returned list as a new column in the dataframe
    link_data['shape_points'] = shape_points
else:
    print("Something went wrong in the shape processing")

[========================================================================] 100%


In [17]:
# calculate all midpoints in link
midpoints = []

for shape in link_data['shape_points']:
    point1 = shape[0]
    point2 = shape[-1]
    midpoints.append(mid_point(point1[0],point1[1],point2[0],point2[1]))

# add new column midpoint to the dataframe
link_data['mid_point'] = midpoints

In [18]:
# calculate distance from probe points to bottom left corner of bounding box
bottom_left_distance = []
bottom_left_coord = (50.62500, 8.43751)

for probe in probe_data.itertuples():
    d = point_distance(probe.latitude,probe.longitude,bottom_left_coord[0],bottom_left_coord[1])
    bottom_left_distance.append(d)

# add new column midpoint_distance to the dataframe
probe_data['probe_distance'] = bottom_left_distance


In [19]:
# calculate distance from mid points to bottom left corner of bounding box
bottom_left_distance = []

for midpoint in link_data['mid_point']:
    d = point_distance(midpoint[0],midpoint[1],bottom_left_coord[0],bottom_left_coord[1])
    bottom_left_distance.append(d)

# add new column midpoint_distance to the dataframe
link_data['midpoint_distance'] = bottom_left_distance

In [20]:
# sorting links by distance from midpoint to bottom left of bounding box
link_data = link_data.sort_values('midpoint_distance')

In [21]:
# map matching
threshold = 50 # This is number of closest links that each probe is going to process
matched_points = map_matching(probe_data,link_data, threshold)

[========================================================================] 100%


In [25]:
print("Writing MATCHED Points to csv! ")
matched_points.to_csv(OUTPUT_MATCHED_PATH,header=True, index=False)

<br></br>
## Determine Direction of Travel

### To calculate position a probe point has travelled after one minute in the given speed and heading

In [22]:
def position_after_one_min(lat,lon,speed,heading):
    # this function takes the current lat and lon position of the probe, speed in KPH, and heading(bearing) of probe

    # convert speed in KPH to KPM and use that as distance travelled in km in one minute
    dist = speed/60
    
    R = 6371.0  # radius of the Earth
    bearing = radians(heading) # convert bearing to radians
    

    lat = radians(52.20472) 
    lon = radians(0.14056) 

    result_lat = asin(sin(lat)*cos(dist/R) + cos(lat)*sin(dist/R)*cos(bearing))

    result_lon = lon + atan2(sin(bearing)*sin(dist/R)*cos(lat),cos(dist/R)-sin(lat)*sin(result_lat))

    result_lat = degrees(result_lat)
    result_lon = degrees(result_lon)

    return (result_lat,result_lon)

In [42]:
def direction_of_travel(matched_points,link_data):
    # This function takes goes through each matched probe point and dtermine the direction of travel for that point
    bar = progressbar.ProgressBar(maxval=probe_data.shape[0], \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    j=0
    
    # Create a copy of the matched_points dataframe as the complete_matched_points output
    complete_matched_points = matched_points.copy()
    
    for probe in matched_points.itertuples():
        start_time = time.time()
        probe_point = (probe.latitude,probe.longitude)
        link_ID = probe.linkPVID
        
        # Look up the ref node lat and lon for this corresponding link in the linkdata dataframe
        link_row = link_data[link_data['linkPVID'] == link_ID]
        link_ref = link_row.shape_points[link_row.index.values[0]][0]

        # Calculate dist between link ref and probe after probe has travelled for one minute
        
        new_probe_position = position_after_one_min(probe.latitude,probe.longitude,probe.speed, probe.heading)
        # Direction for stationary points(speed=0) is N = neither
        if probe.speed==0:
            complete_matched_points.at[probe.Index,'direction']= 'N'
        else:
            dist_after_travel = point_distance(new_probe_position[0],new_probe_position[1],probe_point[0],probe_point[1])
        
        # if distance between probe and ref decreases after travel for 1 min, then it's direction is T = towards ref node
        if dist_after_travel < probe.distFromRef:
            complete_matched_points.at[probe.Index,'direction']= 'T'
        
        j+=1
        bar.update(j)
        end_time = time.time()
        #print(f"Time for {j}: {end_time - start_time}")   
    
    bar.finish()
    return complete_matched_points

In [43]:
complete_matched_points = direction_of_travel(matched_points,link_data)

[========================================================================] 100%


### Finally, multiply values in distFromRef and distFromLink columns by 1000 to get distances in decimal meters

In [44]:
complete_matched_points['distFromRef'] = complete_matched_points['distFromRef'].apply(lambda x: x*1000)
complete_matched_points['distFromLink'] = complete_matched_points['distFromLink'].apply(lambda x: x*1000)

# write to csv
complete_matched_points.to_csv(OUTPUT_MATCHED_PATH_WITH_DIRECTION,header=True, index=False)


### Calculate Slope function

In [168]:
def calculate_slope(matched_points, linkProbePoints):    
    bar = progressbar.ProgressBar(maxval=matched_points.shape[0], \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    # Create dataframe
    slope_calculate=matched_points.copy()
    # Add Slope column
    slope_calculate['slope']=0.0
    # Iterate over matched data
    before_matched_point=None
    index=0
    slope_degree=0.0
    
    for current_matched_point in matched_points.itertuples():

        if before_matched_point==None or current_matched_point.linkPVID!=before_matched_point.linkPVID :
            slope_calculate.at[current_matched_point.Index,'slope']=0.0
        else:
            # Get latitude and longitudes
            lat1=float(current_matched_point.latitude)
            lat2=float(before_matched_point.latitude)
            long1=float(current_matched_point.longitude)
            long2=float(before_matched_point.longitude)
            # Use haversine formula to get distance between two points 
            distance=point_distance(lat1,long1,lat2,long2)
            # Get altitude difference
            altitude=float(current_matched_point.altitude)-float(before_matched_point.altitude)
            # Get slope altitude/distance
            
            try:
                slope=float(altitude/distance)
                # Convert to degrees decimal
                slope_degree=(2*pi*atan(slope))/360
                slope_calculate.at[current_matched_point.Index,'slope']=slope_degree
            except ZeroDivisionError:
                slope_calculate.at[current_matched_point.Index,'slope']=0.0
            
            if current_matched_point.linkPVID in linkProbePoints:
                linkProbePoints[current_matched_point.linkPVID]=max(linkProbePoints.get(current_matched_point.linkPVID), slope_calculate.at[current_matched_point.Index,'slope'])
            else:
                linkProbePoints[current_matched_point.linkPVID]=slope_calculate.at[current_matched_point.Index,'slope']
#             for link in link_data.itertuples():
#                 if link.linkPVID==current_matched_point.linkPVID and link.slopeInfo!='nan':
#                     if link.linkPVID in linkProbePoints:
#                         linkProbePoints[link.linkPVID].append(slope_calculate.at[current_matched_point.Index,'slope'])
#                     else:
#                         linkProbePoints[link.linkPVID]=[]
#                         linkProbePoints[link.linkPVID].append(slope_calculate.at[current_matched_point.Index,'slope'])
#                     break
        
        # Assign it to matched point current
        slope_calculate.at[current_matched_point.Index,'slope']=slope_degree  
        index+=1
        #print(index, end=" ")
        bar.update(index)
        before_matched_point=current_matched_point
    
    bar.finish()
    
    return slope_calculate

### Calculate the slope from matched points

In [170]:
linkProbePoints={}
calculated_slope = calculate_slope(complete_matched_points, linkProbePoints)

# write to csv
print("Writing Slope Data to CSV")
calculated_slope.to_csv(OUTPUT_SLOPE_DATA_PATH,header=True, index=False)

[========================================================================] 100%


Writing Slope Data to CSV


### Evaluate calculated slope with given slope

In [171]:
def calc_avg_slope_matched_points(dataframe, linkID):
    
    size=0
    sum_avg_slope=0.0
    for point in dataframe.itertuples():
        if point.linkPVID==linkID:
            sum_avg_slope+=point.slope
            size+=1
    
    return float(sum_avg_slope/size)

In [172]:
def calc_avg_slope_link_data(dataframe, linkID):
    
    sum_slope_given=0.0
    
    for link in dataframe.itertuples():
            # Calculate average slope from given slope info
        if link.linkPVID==linkID and link.slopeInfo!='nan':
            slope_length_given=len(link.slopeInfo.strip().split('|'))
            for slope_point in link.slopeInfo.strip().split('|'):
                slope_point_split=float(slope_point.strip().split('/')[1])
                sum_slope_given+=slope_point_split
                
            return float(sum_slope_given/slope_length_given)
        else:
            return None
            
    return None

In [ ]:
print("Evaluate slope")
result=pd.DataFrame(data=None, columns=['linkPVID','calculated_slope','given_slope'])
for link in link_data.itertuples():
                    
    uniquePVID=link.linkPVID
    if uniquePVID in linkProbePoints:
        
        # Mean for calculated!
        c_slope=linkProbePoints.get(uniquePVID)
#         c_slope_sum=0.0
#         c_slope_size=0
#         c_slope=0.0
                    
#         for linkProbe in linkProbePoints.get(uniquePVID):
#             if linkProbe.slope!=0.0:
#                 c_slope_sum+=linkProbe.slope
#                 c_slope_size+=1
        
#         if c_slope_size != 0
#             c_slope=float(c_slope_sum/c_slope_size)
        
        # Mean for Given slope
        g_slope_sum=0.0
        g_slope_size=0
        g_slope=0.0       
        
        if link.slopeInfo!='nan':
            for g_slope in link.slopeInfo.strip().split('|'):
                g_slope_sum+=float(g_slope.strip().split('/')[1])
                g_slope_size+=1
        
            if g_slope_size != 0:
                g_slope=float(g_slope_sum/g_slope_size)
        
         ## Write to result dataframe
        result.at[link.Index,'linkPVID']=uniquePVID
        result.at[link.Index, 'given_slope']=g_slope
        result.at[link.Index, 'calculated_slope']=c_slope
    else:
        result.at[link.Index, 'given_slope']=0.0
        result.at[link.Index, 'calculated_slope']=0.0 

# Print this result to csv
print("Writing to Evaluate Slope CSV!")
result.to_csv(EVALUATE_SLOPE_DATA_PATH,header=True, index=False)
    
    
    

Evaluate slope
